In [ ]:
# default_exp core.rq_marketplace

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Marketplace Requests

> Core.rq_marketplace is a collection of function wrappers around the Discogs API for marketplace interactions.

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

## Inventory

In [ ]:
#export


def get_inventory(user: Union[UserWithoutAuthentication,
                              UserWithUserTokenBasedAuthentication], 
                  username: str,
                  status: Union[StatusInventory, None] = None,
                  page: Union[int, None] = None,
                  per_page: Union[int, None] = None,
                  sort: Union[SortOptionsInventory, None] = None,
                  sort_order: Union[SortOrder, None] = None
                  ) -> requests.models.Response:
    """
    Get a list of listings of a given user inventory.
    
    If you are not authenticated as the inventory owner, 
    only items that have a status of "For Sale" will be visible.
    
    No user Authentication needed. 
    
    Parameters:
    
    user: user object (required).
    
    username: string (required) 
        -> The username for whose inventory you are fetching
        
    status: string (optional) 
        -> Only show items with this status.
    
    page: number (optional) 
        -> The page you want to request.
        
    per_page: number (optional)
        -> The number of items per page.

    sort: string (optional)
        -> Sort items by this field.
        
    sort_order: string (optional)
        -> Sort items in a particular order (one of asc, desc)
    
    """
    url = f"{USERS_URL}/{username}/inventory"
    headers = user.headers
    params = user.params
    if status:
        if type(user) is UserWithoutAuthentication:
            params["status"] = StatusInventory.for_sale.value
        else:
            params["status"] = status.value
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if sort:
        if type(user) is UserWithoutAuthentication and (sort is not SortOptionsInventory.status and 
                                                        sort is not SortOptionsInventory.location):
            params["sort"] = sort.name   
        else:
            params["sort"] = sort.name   
    if sort_order:
        params["sort_order"] = sort_order.name
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_inventory
get_inventory(user=example_user, 
              username="Les_nuits_noires", 
              status=StatusInventory.for_sale,
              page=1, 
              per_page=3, 
              sort=SortOptionsInventory.price, 
              sort_order=SortOrder.asc)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_inventory___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{USERS_URL}/user_name/inventory?status=For+Sale&page=1&per_page=3&sort=price&sort_order=asc"
    
    result_content = {"TestInventory": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_inventory(user=test_user, username="user_name", status=StatusInventory.for_sale,
              page=1, per_page=3, sort=SortOptionsInventory.price, sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_inventory___authentication_none()

@responses.activate
def test_get_inventory___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{USERS_URL}/user_name/inventory?token=test_user_token&status=Sold&page=1&per_page=3&sort=price&sort_order=asc"
    
    result_content = {"TestInventory": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_inventory(user=test_user, username="user_name", status=StatusInventory.sold,
              page=1, per_page=3, sort=SortOptionsInventory.price, sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_inventory___authentication_user_token()

## Listing

In [ ]:
#export


def get_listing(user: Union[UserWithoutAuthentication,
                            UserWithUserTokenBasedAuthentication], 
                listing_id: int,
                curr_abbr: Union[CurrAbbr, None] = None
                ) -> requests.models.Response:
    """
    Get all the information about the requested listing from the given user inventory.
    
    No user Authentication needed. 

    Parameters:
    
    user: user object (required).
    
    listing_id: number (required) 
        -> The username for whose inventory you are fetching
        
    curr_abbr: string (optional)
        -> Currency for marketplace data. Defaults to the authenticated users currency.         
    """
    url = f"{LISTINGS_URL}/{listing_id}"
    headers = user.headers
    params = user.params
    if curr_abbr: 
        params["curr_abbr"] = curr_abbr.value
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_listing
get_listing(user=example_user, 
            listing_id=1034876519, 
            curr_abbr=CurrAbbr.EUR)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_listing___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{LISTINGS_URL}/123456?curr_abbr=EUR"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_listing(user=test_user, listing_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_listing___authentication_none()

@responses.activate
def test_get_listing___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LISTINGS_URL}/123456?token=test_user_token&curr_abbr=EUR"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_listing(user=test_user, listing_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_listing___authentication_user_token()

In [ ]:
#export


def update_listing(user: UserWithUserTokenBasedAuthentication, 
                   listing_id: int,
                   release_id: int,
                   condition: ReleaseCondition,
                   price: float,
                   sleeve_condition: Union[SleeveCondition, None] = None,
                   status: Union[StatusInventory, None] = None,
                   comments: Union[str, None] = None,
                   allow_offers: Union[bool, None] = None,
                   external_id: Union[str, None] = None,
                   location: Union[str, None] = None,
                   weight: Union[int, None] = None,
                   format_quantity: Union[int, None] = None,
                   curr_abbr: Union[CurrAbbr, None] = None
                   ) -> requests.models.Response:
    """
    Update the data associated with a listing.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
    
    listing_id: number (required) 
        -> The username for whose inventory you are fetching
        
    release_id: number (required)
        -> The ID of the release you are posting
        
    condition: string (required)
        -> The condition of the release you are posting.
        
    price: number (required)
        -> The price of the item (in the seller’s currency).

    status: string (required)
        -> The status of the listing. Defaults to For Sale. 
        
    sleeve_condition: string (optional)
        -> The condition of the sleeve of the item you are posting. 
        
    comments: string (optional) 
        -> Any remarks about the item that will be displayed to buyers.
        
    allow_offers: boolean (optional)
        -> Whether or not to allow buyers to make offers on the item. Defaults to false.
               
    external_id: string (optional) 
        -> A freeform field that can be used for the seller’s own reference. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field is called “Private Comments” on the Discogs website.
    
    location: string (optional) 
        -> A freeform field that is intended to help identify an item’s physical storage location. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field will be visible on the inventory management page and will be available in inventory exports via the website.
    
    weight: number (optional)
        -> The weight, in grams, of this listing, for the purpose of calculating shipping. 
        -> Set this field to auto to have the weight automatically estimated for you.
    
    format_quantity: number (optional)
        -> The number of items this listing counts as, for the purpose of calculating shipping. 
        -> This field is called “Counts As” on the Discogs website. 
        -> Set this field to auto to have the quantity automatically estimated for you.
    
    curr_abbr: string (optional)
        -> Currency for marketplace data. Defaults to the authenticated users currency.  
    """
        
    url = f"{LISTINGS_URL}/{listing_id}"
    
    params = user.params
    if curr_abbr: 
        params["curr_abbr"] = curr_abbr.value
    
    data = {
        "release_id": release_id,
        "condition": condition.value,
        "price": price,
        "status": status.value,
    }
    
    if sleeve_condition:
        data["sleeve_condition"] = sleeve_condition.value
    if comments:
        data["comments"] = comments
    if allow_offers is not None:
        data["allow_offers"] = allow_offers
    if external_id:
        data["external_id"] = external_id 
    if location:
        data["location"] = location
    if weight:
        data["weight"] = weight 
    if format_quantity:
        data["format_quantity"] = format_quantity

    return requests.post(url, params=params, json=data)

In [ ]:
# Example for usage of update_listing

update_listing(user=example_user,
               listing_id=1059961457,
               release_id=10361797,
               condition=ReleaseCondition.mint,
               price=5.42,
               status=StatusInventory.for_sale,
               sleeve_condition=SleeveCondition.mint,
               comments="CD is new and unplayed",
               allow_offers=False,
               external_id="Private Comment",
               location="Test Location",
               weight=100,
               format_quantity=1,
               curr_abbr=CurrAbbr.EUR) 

<Response [204]>

In [ ]:
#hide

@responses.activate
def test_update_listing___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LISTINGS_URL}/123456?token=test_user_token&curr_abbr=EUR"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=204)

    resp = update_listing(user=test_user,
                          listing_id=123456,
                          release_id=789,
                          condition=ReleaseCondition.mint,
                          sleeve_condition=SleeveCondition.mint,
                          price=10.00,
                          status=StatusInventory.for_sale,
                          comments="Standard CD",
                          allow_offers=False,
                          external_id="Private Comment",
                          location="Test Location",
                          weight=100,
                          format_quantity=1,
                          curr_abbr=CurrAbbr.EUR) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_update_listing___authentication_user_token()


In [ ]:
#export


def delete_listing(user: UserWithUserTokenBasedAuthentication, 
                   listing_id: int
                   ) -> requests.models.Response:
    """
    Delete a given listing from the inventory.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
    
    listing_id: number (required) 
        -> The username for whose inventory you are fetching.
    """
        
    url = f"{LISTINGS_URL}/{listing_id}"
    headers = user.headers  
    params = user.params

    return requests.delete(url, headers=headers, params=params)

In [ ]:
# Example for usage of delete_listing

delete_listing(user=example_user, 
               listing_id=1052411061)

<Response [204]>

In [ ]:
#hide

@responses.activate
def test_delete_listing___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LISTINGS_URL}/123456?token=test_user_token"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.DELETE, target_url,
                  json=result_content, status=204)

    resp = delete_listing(user=test_user, 
                          listing_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_delete_listing___authentication_user_token()

In [ ]:
#export


def add_new_listing(user: UserWithUserTokenBasedAuthentication, 
                    release_id: int,
                    condition: ReleaseCondition,
                    price: float,
                    status: StatusNewListing,
                    sleeve_condition: Union[SleeveCondition, None] = None,
                    comments: Union[str, None] = None,
                    allow_offers: Union[bool, None] = None,
                    external_id: Union[str, None] = None,
                    location: Union[str, None] = None,
                    weight: Union[int, None] = None,
                    format_quantity: Union[int, None] = None
                    ) -> requests.models.Response:
    """
    Add a new listing to the given user inventory.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
               
    release_id: number (required)
        -> The ID of the release you are posting
        
    condition: string (required)
        -> The condition of the release you are posting.
        
    price: number (required)
        -> The price of the item (in the seller’s currency).

    status: string (required)
        -> The status of the listing. Defaults to For Sale. 
    
    sleeve_condition: string (optional)
        -> The condition of the sleeve of the item you are posting.     
    
    comments: string (optional) 
        -> Any remarks about the item that will be displayed to buyers.
        
    allow_offers: boolean (optional)
        -> Whether or not to allow buyers to make offers on the item. Defaults to false.
               
    external_id: string (optional) 
        -> A freeform field that can be used for the seller’s own reference. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field is called “Private Comments” on the Discogs website.
    
    location: string (optional) 
        -> A freeform field that is intended to help identify an item’s physical storage location. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field will be visible on the inventory management page and will be available in inventory exports via the website.
    
    weight: number (optional)
        -> The weight, in grams, of this listing, for the purpose of calculating shipping. 
        -> Set this field to auto to have the weight automatically estimated for you.
    
    format_quantity: number (optional)
        -> The number of items this listing counts as, for the purpose of calculating shipping. 
        -> This field is called “Counts As” on the Discogs website. 
        -> Set this field to auto to have the quantity automatically estimated for you.
    """
        
    url = f"{LISTINGS_URL}"
    headers = user.headers
    
    params = user.params
   
    data = {
        "release_id": release_id,
        "condition": condition.value,
        "price": price,
        "status": status.value,
    }
    
    if sleeve_condition:
        data["sleeve_condition"] = sleeve_condition.value
    if comments:
        data["comments"] = comments
    if allow_offers is not None:
        data["allow_offers"] = allow_offers
    if external_id:
        data["external_id"] = external_id 
    if location:
        data["location"] = location
    if weight:
        data["weight"] = weight 
    if format_quantity:
        data["format_quantity"] = format_quantity
    
    return requests.post(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of add_new_listing

add_new_listing(user=example_user,
                release_id=10361797,
                condition=ReleaseCondition.mint,
                price=10.00,
                status=StatusNewListing.draft,
                sleeve_condition=SleeveCondition.mint,
                comments="Standard CD",
                allow_offers=False,
                external_id="Private Comment",
                location="Test Location",
                weight=100,
                format_quantity=1) 

<Response [201]>

In [ ]:
#hide

@responses.activate
def test_add_new_listing___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LISTINGS_URL}?token=test_user_token"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=201)

    resp = add_new_listing(user=test_user,
                           release_id=123456,
                           condition=ReleaseCondition.mint,
                           price=10.00,
                           status=StatusNewListing.draft,
                           sleeve_condition=SleeveCondition.mint,
                           comments="Standard CD",
                           allow_offers=False,
                           external_id="Private Comment",
                           location="Test Location",
                           weight=100,
                           format_quantity=1) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_add_new_listing___authentication_user_token()


## Order

In [ ]:
#export


def get_order(user: UserWithUserTokenBasedAuthentication, 
              order_id: str
              ) -> requests.models.Response:
    """
    Get the data associated with the given order.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
               
    order_id: string (required)
        -> The ID of the order you are fetching
    """
    url = f"{ORDERS_URL}/{order_id}"
    headers = user.headers
    params = user.params
    
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_order

get_order(user=example_user, 
          order_id="1906948-512") 

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_order___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}/123456?token=test_user_token"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_order(user=test_user, order_id="123456") 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_order___authentication_user_token()
   

In [ ]:
#export


def update_order_status(user: UserWithUserTokenBasedAuthentication, 
                        order_id: str,
                        status: StatusOrder
                        ) -> requests.models.Response:
    """
    Update the status associated with the given order.
    
    Note: Use get_order to get the next_status key. 
          The next_status key is an array of valid next statuses for this order.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
               
    order_id: string (required)
        -> The ID of the order you are fetching
        
    status: string (required)
        -> The status of the Order you are updating. 
    """
    url = f"{ORDERS_URL}/{order_id}"
    headers = user.headers
    params = user.params
    
    data = {"status": status.value}

    return requests.post(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of update_order_status

update_order_status(user=example_user, 
                    order_id="1906948-522", 
                    status=StatusOrder.payment_received) 

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_update_order_status___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}/123456?token=test_user_token"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=200)

    resp = update_order_status(user=test_user, order_id="123456", status=StatusOrder.invoice_sent) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_update_order_status___authentication_user_token()

In [ ]:
#export


def update_order_shipping(user: UserWithUserTokenBasedAuthentication, 
                          order_id: str,
                          shipping_cost: float
                          ) -> requests.models.Response:
    """
    Update the shipping costs associated with the given order.
    
    Note: If the order status is neither cancelled, Payment Received, nor Shipped, you can change the shipping. 
          Doing so will send an invoice to the buyer and set the order status to Invoice Sent.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
               
    order_id: string (required)
        -> The ID of the order you are fetching
        
    shipping_cost: number (required)
        -> The order shipping amount. 
    """
    url = f"{ORDERS_URL}/{order_id}"
    headers = user.headers
    params = user.params
    
    data = {"shipping": shipping_cost}

    return requests.post(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of update_order_shipping

update_order_shipping(user=example_user, 
                      order_id="1906948-516", 
                      shipping_cost=3.70) 

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_update_order_shipping___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}/123456?token=test_user_token"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=200)

    resp = update_order_shipping(user=test_user, order_id="123456", shipping_cost=3.70) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_update_order_shipping___authentication_user_token()

In [ ]:
#export


def get_orders(user: UserWithUserTokenBasedAuthentication,
               status: Union[StatusOrders, None] = None,
               page: Union[int, None] = None,
               per_page: Union[int, None] = None,
               sort: Union[SortOptionsOrders, None] = None,
               sort_order: Union[SortOrder, None] = None
               ) -> requests.models.Response:
    """
    Get a list of orders associated with the given user.

    Note: In the Discogs API is an optional parameter called "archived".
          I did not implement it because it always caused a server-sided crash.
          
    User Authentication needed.  
    
    Parameters:
    
    user: user object (required).
               
    status: string (optional) 
        -> Only show items with this status.
    
    page: number (optional) 
        -> The page you want to request.
        
    per_page: number (optional)
        -> The number of items per page.

    sort: string (optional)
        -> Sort items by this field.
        
    sort_order: string (optional)
        -> Sort items in a particular order (one of asc, desc)
    """
    url = f"{ORDERS_URL}"
    headers = user.headers
    params = user.params

    if status:
        params["status"] = status.value
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if sort:
        params["sort"] = sort.name   
    if sort_order:
        params["sort_order"] = sort_order.name
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_orders

get_orders(user=example_user, 
           status=StatusOrders.payment_pending, 
           page=1, 
           per_page=3, 
           sort=SortOptionsOrders.created, 
           sort_order=SortOrder.asc) 

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_orders___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}?token=test_user_token&status=Payment+Pending&page=1&per_page=3&sort=created&sort_order=asc"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_orders(user=test_user, status=StatusOrders.payment_pending, page=1, 
                      per_page=3, sort=SortOptionsOrders.created, sort_order=SortOrder.asc) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_orders___authentication_user_token()

In [ ]:
#export


def get_order_messages(user: UserWithUserTokenBasedAuthentication,
                       order_id: str,
                       page: Union[int, None] = None,
                       per_page: Union[int, None] = None
                       ) -> requests.models.Response:
    """
    Get a list of messages associated with the given order, with the most recent first.
          
    User Authentication needed.   
    
    Parameters:
    
    user: user object (required)
               
    order_id: string (required) 
        -> The ID of the order you are fetching
    
    page: number (optional) 
        -> The page you want to request.
        
    per_page: number (optional)
        -> The number of items per page.
    """
    url = f"{ORDERS_URL}/{order_id}/messages"
    headers = user.headers
    params = user.params

    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)

    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_order_messages

get_order_messages(user=example_user, 
                   order_id="1906948-515", 
                   page=1, 
                   per_page=3) 

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_order_messages___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}/123456/messages?token=test_user_token&page=1&per_page=3"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_order_messages(user=test_user, order_id="123456", page=1, per_page=3) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_order_messages___authentication_user_token()


In [ ]:
#export


def post_message_on_order(user: UserWithUserTokenBasedAuthentication,
                          order_id: str,
                          message: str,
                          ) -> requests.models.Response:
    """
    Post a new message to the given order.
          
    User Authentication needed.   
    
    Parameters:
    
    user: user object (required)
               
    order_id: string (required) 
        -> The ID of the order you are fetching
    
    message: string (required) 
    """
    url = f"{ORDERS_URL}/{order_id}/messages"
    headers = user.headers
    params = user.params

    data = {
        "message": message
    }

    return requests.post(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of post_message_on_order

post_message_on_order(user=example_user, 
                      order_id="1906948-516", 
                      message="Hello World") 

<Response [201]>

In [ ]:
#hide

@responses.activate
def test_post_message_on_order___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ORDERS_URL}/123456/messages?token=test_user_token"
    
    result_content = {"TestOrder": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=201)

    resp = post_message_on_order(user=test_user, order_id="123456", message="Hello World") 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_post_message_on_order___authentication_user_token()

## Fee

In [ ]:
#export


def get_discogs_fee_for_given_price(user: Union[UserWithoutAuthentication,
                                                UserWithUserTokenBasedAuthentication], 
                                    price: float
                                    ) -> requests.models.Response:
    """
    Calculate the fee for selling an item on the Marketplace in USD
    
    Note: According to the Discogs API, it should be possible to change the currency. 
          However, this feature seems to be broken at this point. 
          That's why I did remove this optional argument.
    
    No user Authentication needed. 

    Parameters:
    
    user: user object (required)
               
    price: number (required) 
        -> The price to calculate a fee from
    """
    
    url = f"{FEE_URL}/{price}"
    headers = user.headers
    params = user.params

    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_discogs_fee_for_given_price

get_discogs_fee_for_given_price(user=example_user, 
                                price=5.42) 


<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_discogs_fee_for_given_price___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{FEE_URL}/666"
    
    result_content = {"TestFee": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_discogs_fee_for_given_price(user=test_user, price=666) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_discogs_fee_for_given_price___authentication_none()

@responses.activate
def test_get_discogs_fee_for_given_price___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{FEE_URL}/666?token=test_user_token"
    
    result_content = {"TestFee": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_discogs_fee_for_given_price(user=test_user, price=666) 

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_discogs_fee_for_given_price___authentication_user_token()


## Price Suggestions

In [ ]:
#export


def get_price_suggestions(user: UserWithUserTokenBasedAuthentication,
                          release_id: int
                          ) -> requests.models.Response:
    """
    Get a price suggestions for the given release id. 
    
    Note: Suggested prices will be denominated in the user’s selling currency.
          If no suggestions are available, an empty object will be returned.
    
    User Authentication needed. 

    Parameters:
    
    user: user object (required)
               
    release_id: number (required) 
        -> The release ID to calculate a price from.
    """
    
    url = f"{PRICE_SUGGESTIONS_URL}/{release_id}"
    headers = user.headers
    params = user.params

    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_price_suggestions

get_price_suggestions(user=example_user, 
                      release_id=9453891) 


Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.rq_database.ipynb.
Converted 03_core.rq_marketplace.ipynb.
Converted 04_core.rq_inventory_export.ipynb.
Converted 05_core.rq_inventory_upload.ipynb.
Converted 06_core.rq_collection.ipynb.
Converted 07_core.rq_wantlist.ipynb.
Converted index.ipynb.
